# TensorFlow - Embeddings Viz

In [1]:
# importing packages
import tensorflow as tf
import numpy as np
import pandas as pd
import io
import csv

from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### Get IMdb data

In [2]:
# reading 50,000 movie reviews labeled as "positive" or "negative"
imdb_data = pd.read_csv("./data/IMDB Dataset.csv")

### Preview Dataframe

In [3]:
# checking first rows
imdb_data.loc[0:5]

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive


### Transform categorical target to binary

In [4]:
# processing data
imdb_data["sentiment"] = imdb_data["sentiment"].map({"positive": 1, "negative":0})

In [5]:
# checking first rows
imdb_data.loc[0:5]

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
5,"Probably my all-time favorite movie, a story o...",1


### Train/Test Split

In [6]:
# defining train and test data sets
imdb_train = imdb_data["review"].values[:25000]
imdb_test = imdb_data["review"].values[25000:]
imdb_train_labels = imdb_data["sentiment"].values[:25000]
imdb_test_labels = imdb_data["sentiment"].values[25000:]

In [7]:
imdb_train[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [8]:
imdb_train_labels[0]

1

### Hyperparameters

In [9]:
# listing hyperparameters makes it easier to quickly test different combinations
vocab_size = 10000
embedding_dim = 16
# a sentence shorter than max_length it will be padded, longer sentences will be truncated
max_length = 120  

### Tokenize

In [10]:
# tokenizing data
tokenizer = Tokenizer(num_words=10000, oov_token = "<oov>")
tokenizer.fit_on_texts(imdb_train)
word_index = tokenizer.word_index

# converting words to numbers and pad for the neural network to use as input
train_sequences = tokenizer.texts_to_sequences(imdb_train)
train_padded = pad_sequences(train_sequences, maxlen=120, truncating="post")

# tokenized using the word_index learned from the training data
testing_sequences = tokenizer.texts_to_sequences(imdb_test)
test_padded = pad_sequences(testing_sequences, maxlen=120, truncating="post")

### Compare Tokenization

In [11]:
# reversing keys: keys become the values, and values become the keys so that we can look a word up
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

# this is what will be fed in
print(decode_review(train_padded[1]))

# this is the original text
print(imdb_train[1])

a wonderful little production br br the filming technique is very <oov> very old time bbc fashion and gives a <oov> and sometimes <oov> sense of realism to the entire piece br br the actors are extremely well chosen michael sheen not only has got all the <oov> but he has all the voices down pat too you can truly see the <oov> editing guided by the references to <oov> diary entries not only is it well worth the watching but it is a <oov> written and performed piece a masterful production about one of the great <oov> of comedy and his life br br the realism really comes home with the little things the fantasy of the guard which
A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seam

### Create Embeddings

In [12]:
# defining model
model = tf.keras.Sequential([
    # the "embedding layer" is the key to text sentiment analysis
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.fit(train_padded, imdb_train_labels, epochs=10, validation_data=(test_padded, imdb_test_labels))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 2s 92us/sample - loss: 0.4918 - accuracy: 0.7457 - val_loss: 0.3601 - val_accuracy: 0.8376
Epoch 2/10
25000/25000 [==============================] - 2s 83us/sample - loss: 0.2433 - accuracy: 0.9046 - val_loss: 0.3842 - val_accuracy: 0.8322
Epoch 3/10
25000/25000 [==============================] - 2s 92us/sample - loss: 0.0942 - accuracy: 0.9755 - val_loss: 0.4783 - val_accuracy: 0.8230
Epoch 4/10
25000/25000 [==============================] - 2s 89us/sample - loss: 0.0244 - accuracy: 0.9964 - val_loss: 0.5603 - val_accuracy: 0.8234
Epoch 5/10
25000/25000 [==============================] - 2s 89us/sample - loss: 0.0063 - accuracy: 0.9996 - val_loss: 0.6170 - val_accuracy: 0.8240
Epoch 6/10
25000/25000 [==============================] - 2s 79us/sample - loss: 0.0022 - accuracy: 0.9999 - val_loss: 0.6759 - val_accuracy: 0.8246
Epoch 7/10
25000/25000 [==============================] 

### Embeddings

In [14]:
# output from the "embedding layer"
embeddings = model.layers[0]
weights = embeddings.get_weights()[0]
print(f"Vocabulary size: {weights.shape[0]},  Embedding dimensions: {weights.shape[1]}")

Vocabulary size: 10000,  Embedding dimensions: 16


In [15]:
# getting the shape
weights.shape

(10000, 16)

In [16]:
weights[0]

array([ 0.0080015 , -0.05046177, -0.07983058, -0.03529545,  0.03482727,
        0.01164579,  0.02964137, -0.0471835 , -0.03701125,  0.03281684,
       -0.00434467, -0.00110458, -0.02429215, -0.01236048, -0.00131568,
        0.03766251], dtype=float32)

### Visualize the embedding vectors

In [17]:
# writing the vectors and their metadata out to file. 
# these 2 files ('vecs.tsv', 'meta.tsv') are used by the 
# TensorFlow projector (http://projector.tensorflow.org/)
# to plot/visualize the vectors/embeddings in 3D

# output of the 16 values per word representation (embedding)
#      out_v are the vector/weights (embedding)
#      out_m are the actual words (meta data) associated with each embedding

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [18]:
# reading it
with open("vecs.tsv") as file:
    rd = csv.reader(file, delimiter="\t", quotechar='"')
    for row in rd:
        print(row)

['-0.015336267', '-0.06639506', '-0.13827151', '-0.082461484', '0.025598088', '-0.032763515', '-0.008228521', '-0.049470637', '-0.05495559', '0.0013722873', '-0.05385989', '-0.010763746', '-0.01575991', '-0.059347466', '-0.013754611', '-0.047797125']
['0.027361248', '-0.052628897', '-0.17149718', '-0.02483928', '0.08487196', '-0.035360772', '-0.006227229', '-0.034810685', '-0.05501937', '-0.002932519', '-0.028589623', '-0.0009475991', '-0.02830173', '-0.11819167', '-0.015736662', '-0.078305885']
['-0.05766761', '-0.018583544', '-0.088770434', '-0.086951874', '0.12864392', '0.021837384', '0.009344074', '0.015913721', '-0.057747617', '0.009788367', '-0.07366293', '0.02162416', '0.016911088', '-0.08601349', '0.017036825', '-0.022786066']
['-0.0065573542', '-0.05798188', '-0.17433587', '0.0082151005', '0.04624924', '-0.046607945', '-0.0010474024', '-0.0062163845', '0.003142837', '0.014498899', '-0.14830072', '0.071299136', '0.032858886', '-0.13654129', '-0.062800184', '-0.06460836']
['0.04

['0.0055092573', '0.085628584', '0.08602874', '-0.054938097', '0.07703341', '0.012735082', '-0.0980875', '-0.035834003', '0.075061046', '-0.121103875', '-0.061775114', '0.11096592', '0.015837079', '-0.080993295', '-0.17010778', '0.08257767']
['-0.057182103', '-0.031404857', '-0.14773335', '-0.03707908', '0.09409308', '0.016791167', '-0.08469667', '0.2405236', '0.058472313', '-0.037078824', '-0.16932125', '0.06278329', '-0.025998572', '-0.01266412', '0.025184954', '-0.1029778']
['-0.016118387', '-0.006340863', '-0.09053656', '-0.026396966', '0.027953103', '0.107965015', '-0.013305144', '-0.08236501', '-0.083592415', '-0.10742611', '0.13065957', '0.059426017', '-0.04141981', '-0.05985714', '-0.11014088', '-0.0041704974']
['0.034068093', '-0.05435414', '0.0009871977', '-0.034761246', '0.07709243', '-0.08734088', '0.049165398', '0.030238578', '-0.16558743', '-0.060682576', '0.07863752', '0.051967416', '0.026708093', '-0.10455496', '-0.05611459', '-0.015036486']
['-0.029841635', '-0.1368482

['-0.02861863', '0.04974753', '-0.0081439875', '-0.09730597', '0.07818749', '0.04210803', '0.13842589', '0.038162485', '-0.14792329', '-0.08220968', '0.047947563', '0.061844822', '-0.030680679', '-0.05839079', '0.028401628', '0.068288445']
['-0.026081067', '-0.10644001', '-0.08952163', '0.055197593', '-0.06116712', '-0.11775831', '0.030778445', '-0.07832322', '-0.0072030844', '0.08871514', '0.15662113', '0.003715144', '0.0010824888', '0.09828128', '0.008949117', '0.032172676']
['-0.03730949', '0.052970175', '-0.08420883', '0.06521468', '0.07021598', '0.16469833', '-0.04321793', '0.15967993', '-0.07286439', '-0.031736277', '-0.090545334', '-0.021932924', '0.02885452', '0.017961115', '-0.047586948', '0.14894144']
['-0.18079282', '-0.08054403', '-0.10389091', '-0.024325058', '-0.010870566', '0.054524966', '-0.026775062', '0.061789166', '0.07096991', '-0.053376127', '-0.13613823', '0.052066635', '-0.05637232', '-0.019634709', '-0.033563163', '0.005110795']
['-0.10834001', '0.033007577', '-

['-0.08422518', '-0.10305154', '0.1107667', '-0.034860663', '0.025312351', '-0.041150007', '-0.070726246', '-0.12067193', '-0.015964607', '0.08736714', '-0.013031149', '0.049436957', '0.03571472', '0.066390224', '-0.018881593', '0.10727504']
['0.093317136', '-0.1063147', '-0.078062765', '-0.07011807', '-0.12627983', '-0.13776211', '0.0932616', '-0.077033445', '-0.020484399', '0.14060606', '-0.067796804', '-0.14045529', '0.057240706', '-0.011553929', '0.15135601', '-0.15434419']
['-0.16411406', '0.025538702', '0.19360621', '0.10076909', '-0.009563419', '0.051331066', '-0.1117306', '-0.059877984', '-0.06919998', '-0.09417753', '0.2237158', '-0.09688772', '0.051659364', '0.17929903', '-0.07417974', '0.057530183']
['0.099502824', '0.03232394', '-0.1345777', '0.017062677', '-0.1629302', '-0.17273119', '0.057976544', '-0.023364956', '-0.14290555', '0.16984874', '0.054455232', '-0.051571965', '-0.015737778', '0.016199633', '0.048658118', '-0.108511984']
['-0.03667699', '-0.021654312', '0.0412

['-0.113968536', '0.033016834', '-0.09802333', '0.09040574', '0.04837921', '-0.012423834', '-0.0024311258', '0.11179982', '0.06349547', '0.013051989', '0.044227354', '0.13003711', '0.08864879', '-0.03482813', '-0.045836348', '-0.0039781462']
['0.003085083', '0.034123413', '-0.09177551', '-0.02212256', '0.033699475', '0.068997234', '-0.12823404', '0.06701422', '0.070891514', '0.11188983', '0.09173147', '-0.0062253308', '0.045467414', '-0.05865382', '-0.047908615', '-0.08841857']
['0.051517773', '-0.017158814', '0.10382363', '-0.050641324', '0.09819613', '-0.04375134', '-0.01246809', '0.004477622', '0.17236638', '-0.050241847', '0.009065179', '0.046245985', '-0.031303976', '-0.00808118', '-0.11135662', '0.050714977']
['0.057730433', '-0.06751904', '-0.14153981', '0.04767279', '-0.03909712', '-0.01932962', '-0.042102464', '-0.03505298', '0.031810623', '0.005238067', '0.09273377', '-0.03263226', '-0.07128475', '-0.13770887', '-0.100124486', '0.018847939']
['-0.14613825', '0.030379418', '0.

...

['-0.05550669', '0.013080334', '-0.19228408', '0.048359558', '-0.00064933655', '0.027477764', '0.14591335', '-0.1745511', '-0.017078433', '-0.07479363', '-0.15731473', '-0.08655691', '-0.036296953', '-0.12841901', '0.034877352', '-0.02376963']
['0.033511445', '-0.07074284', '0.120584264', '0.00020059945', '0.02839147', '-0.042268462', '0.010796569', '0.010798198', '-0.01511125', '-0.09660622', '0.024987997', '0.017992945', '-0.140203', '0.08796843', '0.033719223', '-0.048851226']
['-0.0127015775', '-0.012104921', '-0.02624864', '-0.06888523', '0.0033854372', '-0.0046476354', '-0.099888414', '0.038896315', '-0.011500701', '-0.0075039756', '0.007131587', '-0.035991903', '0.09855389', '-0.015622911', '-0.12358753', '0.026441215']
['0.07453861', '-0.13724951', '0.07498282', '0.019199243', '-0.06785695', '-0.03270294', '0.044773787', '-0.030404912', '-0.105588414', '0.21992686', '0.09315813', '-0.050804347', '-0.05113232', '-0.11582714', '0.111315854', '-0.18696712']
['-0.06750783', '0.0452

['0.11286883', '-0.022075312', '-0.16511244', '0.09988396', '-0.06481414', '-0.085705504', '0.1321502', '-0.06063763', '-0.11558804', '0.11761961', '-0.09206265', '-0.052298944', '-0.0971337', '-0.035284366', '0.07261874', '-0.09241512']
['-0.0013087887', '0.036393184', '-0.14457795', '0.036624555', '0.03778713', '0.02241542', '-0.030912183', '0.11445003', '0.019155223', '-0.045307625', '0.027995894', '-0.025364961', '-0.08807545', '0.084495775', '-0.060334373', '-0.02044952']
['-0.034521013', '-0.0077174995', '0.033923216', '0.0699221', '-0.034909338', '0.0063032997', '-0.019133754', '0.033416867', '-0.037421674', '0.09083428', '-0.08074775', '0.0048765615', '0.09934137', '0.0046657813', '0.15051764', '-0.13901073']
['0.11894755', '-0.06228883', '-0.13501446', '0.09120652', '-0.06526747', '-0.13512765', '0.1522929', '-0.0023230985', '0.09900472', '0.09037665', '-0.14422898', '-0.119213834', '-0.017949836', '-0.1482324', '0.116306454', '-0.06218269']
['0.062617', '-0.17049399', '-0.027

['-0.03615919', '0.087924406', '-0.031716887', '-0.10004628', '0.11260232', '0.06516466', '-0.052712966', '0.122865826', '0.16054711', '-0.0708059', '0.09338994', '0.12798075', '-0.07377312', '-0.012101892', '0.06533498', '-0.00911758']
['0.054041665', '0.016403498', '-0.1403451', '0.030252267', '-0.041648928', '0.06069064', '-0.01744381', '-0.0067696474', '-0.1651751', '-0.04439958', '0.081310816', '0.011959803', '-0.060655057', '0.03359931', '0.042194318', '-0.106318876']
['0.03393075', '-0.06949426', '0.047218945', '0.09523492', '-0.033837818', '-0.24913761', '0.015941601', '0.037345123', '0.027192585', '0.06522151', '-0.031653922', '-0.0283314', '-0.07199127', '-0.03821929', '0.063486375', '-0.13862123']
['0.07675172', '-0.12347355', '-0.14828989', '0.0078926915', '-0.06080336', '0.009240089', '0.0798152', '-0.07956354', '0.08790045', '-0.022471458', '0.1053748', '-0.06105637', '-0.15281714', '-0.043499272', '0.059752293', '-0.0539969']
['0.005893248', '-0.027532177', '-0.14899187'

['0.02315479', '0.02510722', '0.0042914352', '0.03860431', '0.031541016', '0.008514631', '-0.111172214', '-0.043990847', '-0.0102862315', '0.06505344', '0.09539499', '0.07400545', '-0.03337918', '-0.02597891', '-0.09714229', '0.03135565']
['0.10638425', '-0.0073026535', '-0.08494527', '0.03727331', '-0.0601203', '-0.10672845', '0.033591468', '0.034393992', '-0.021997092', '-0.11945546', '-0.13219881', '-0.053977236', '-0.14455792', '0.039401297', '-0.010771454', '-0.09708296']
['0.04736821', '-0.09132722', '0.1892613', '-0.020904435', '-0.03256498', '0.0024273836', '0.050222337', '-0.052281093', '-0.1622742', '-0.0851388', '0.22707301', '-0.02656352', '0.048144083', '0.10053791', '0.022660634', '0.061581276']
['0.025430612', '-0.13673477', '-0.16167209', '0.07158914', '-0.07532282', '-0.09685619', '0.053510573', '-0.046605594', '-0.008389136', '0.1403448', '-0.039021976', '-0.107325934', '-0.018346969', '0.17388152', '0.051956233', '-0.08931457']
['0.09444886', '-0.09219069', '0.068875

['-0.014241839', '0.029808179', '-0.08795613', '-0.11583402', '0.044374466', '0.006400181', '-0.018216008', '0.07359542', '-0.042162195', '0.10593659', '0.13713565', '0.07250026', '0.19069777', '-0.018542321', '-0.12979588', '0.0051933182']
['0.092238', '-0.011428959', '-0.21609057', '0.026544552', '0.006823914', '-0.034612473', '0.007734223', '0.092968695', '-0.0054471637', '0.04140368', '0.094453834', '-0.035261', '-0.014786196', '0.20436102', '0.081767045', '0.044983037']
['-0.00746589', '0.07614371', '0.074660026', '-0.055235233', '0.08552276', '0.017990714', '-0.09304674', '-0.031239651', '0.024542429', '-0.050738774', '0.05236919', '0.026469234', '0.016362267', '-0.057999723', '-0.06727293', '0.11854024']
['-0.062307853', '0.06530032', '-0.03186507', '0.0064274888', '-0.026867025', '-0.0050753476', '-0.05083352', '-0.081515975', '-0.17244965', '-0.024450805', '0.0077411514', '-0.010133043', '-0.06212351', '-0.10342567', '0.027376514', '-0.0045152893']
['0.075697', '-0.08512757', 

In [19]:
# reading meta.tsv
with open("meta.tsv") as fd:
    rd = csv.reader(fd, delimiter="\t", quotechar='"')
    for row in rd:
        print(row)

['<oov>']
['the']
['and']
['a']
['of']
['to']
['is']
['br']
['in']
['it']
['i']
['this']
['that']
['was']
['as']
['with']
['for']
['movie']
['but']
['film']
['on']
['not']
['you']
['are']
['his']
['have']
['be']
['one']
['he']
['all']
['at']
['by']
['an']
['they']
['so']
['who']
['from']
['like']
['or']
['just']
['her']
['about']
['out']
['if']
["it's"]
['has']
['there']
['some']
['what']
['good']
['more']
['when']
['very']
['up']
['no']
['time']
['even']
['my']
['would']
['she']
['which']
['only']
['story']
['their']
['see']
['really']
['can']
['had']
['well']
['me']
['were']
['than']
['much']
['we']
['get']
['been']
['bad']
['other']
['will']
['do']
['because']
['people']
['also']
['great']
['into']
['how']
['most']
['first']
['him']
["don't"]
['made']
['make']
['its']
['them']
['then']
['way']
['movies']
['too']
['any']
['could']
['after']
['think']
['characters']
['films']
['watch']
['many']
['two']
['seen']
['character']
['being']
['never']
['acting']
['plot']
['best']
['love']
['

['available']
['leaving']
['ray']
['gang']
['twists']
['failed']
['explain']
['normal']
['opera']
['naked']
['journey']
['jim']
['standard']
['appearance']
['personally']
['utterly']
['historical']
['constantly']
['mary']
['shooting']
['bottom']
['lovely']
['christmas']
['beautifully']
['animated']
['smart']
['natural']
['narrative']
['likely']
['park']
['edge']
['producer']
['subtle']
['cinematic']
['scenery']
['appeal']
['conclusion']
['door']
['chemistry']
['spoiler']
['presented']
['contains']
['terms']
['bother']
['planet']
['sam']
['giant']
['wild']
['terrific']
['changes']
['suppose']
['frank']
['adaptation']
['taste']
['complex']
['g']
['zombies']
['managed']
['steve']
['week']
['costumes']
['bond']
['charming']
['smith']
['emotion']
['innocent']
['rare']
['jones']
['catch']
['wow']
['key']
['places']
['emotions']
['attack']
['mood']
['motion']
['disappointing']
['government']
['fully']
['presence']
['slasher']
['outstanding']
['drugs']
['everybody']
['track']
['generally']
['d

['grows']
['1950s']
['broadcast']
['striking']
['indians']
['trade']
['afterwards']
['killings']
['neat']
['pushed']
['furthermore']
['achievement']
['cardboard']
['aimed']
['receive']
['assassin']
['birthday']
['shirt']
['exaggerated']
['exercise']
['wake']
['generic']
['reasonably']
['bird']
['expert']
['thrills']
['cringe']
['segments']
['affected']
['relevant']
['inspiring']
['shower']
['extended']
['factory']
['hatred']
['strike']
['depression']
['shall']
['clues']
['interaction']
['travels']
['terrifying']
['comics']
['distance']
['ranks']
["world's"]
['threat']
['represents']
['presumably']
['dragged']
['opinions']
['freeman']
['complain']
['allowing']
['emma']
['brilliance']
['gradually']
['pseudo']
['shortly']
['gothic']
['investigation']
['murray']
['messages']
['swear']
['clown']
['demands']
['strikes']
['superbly']
['carl']
['diane']
['primarily']
['virus']
['explosions']
['cabin']
['shoes']
['oscars']
['canada']
['foster']
['blowing']
['nearby']
['corpse']
['requires']
['w

['affection']
['suggested']
['illogical']
['fascinated']
['activities']
['disliked']
['elvis']
['glasses']
['overwhelming']
['experiments']
['imitation']
['operation']
['directs']
['weakest']
['civilization']
['dropping']
['incompetent']
['disappoint']
['advertising']
['graham']
['simplicity']
['muppet']
['nomination']
['elvira']
['spain']
['jeffrey']
['occasion']
['dutch']
['couples']
['inappropriate']
['sarandon']
['conscious']
['betrayal']
['rourke']
['neighbors']
['miniseries']
['lowest']
['stilted']
['complaints']
['desires']
['natalie']
['twelve']
['unhappy']
['mute']
['appreciation']
['visiting']
['karloff']
['frost']
['intentionally']
['rude']
['pitiful']
['mail']
['premiere']
['widescreen']
['perry']
['stranded']
['introduce']
['el']
['vaguely']
['subplots']
['julian']
['remarkably']
['schools']
['troops']
["year's"]
['harrison']
['resemblance']
['rocket']
['solo']
['backgrounds']
['challenged']
['3rd']
['rising']
['bruno']
['houses']
['nicholson']
['trend']
['experimental']
[

['expects']
['depths']
['ie']
['stretched']
['dont']
['resident']
['champion']
['scifi']
['restored']
['harold']
['outfits']
['mentality']
['icon']
['bow']
['cannon']
['conviction']
["1960's"]
['sydney']
['husbands']
['montana']
['pfeiffer']
['macho']
['unreal']
['posted']
['screw']
['conveys']
['sung']
['punches']
['respectable']
['disgusted']
['ruby']
['cope']
['bars']
['coincidence']
['chills']
['1936']
['attacking']
['joins']
['sending']
['rochester']
['assignment']
['subtly']
['spree']
['fascination']
['shortcomings']
['visits']
['polished']
['heights']
['pornography']
['1976']
['empathy']
['amused']
['leo']
['realised']
['reader']
['towers']
['interests']
['fifty']
['capital']
['debbie']
['pit']
['strings']
['1974']
['tacky']
['describing']
['wholly']
['carlos']
['heading']
['kay']
['edit']
['arguably']
['heartfelt']
['acclaimed']
['communicate']
['narrow']
['vastly']
['en']
['witnesses']
['signed']
['kapoor']
['aaron']
['gus']
['sunny']
["husband's"]
['modesty']
['lackluster']
[

['jo']
['timed']
['sundance']
['swallow']
['havoc']
['seasoned']
['agency']
['advertised']
['stream']
['weaker']
['jolie']
['dominated']
['shockingly']
['clouds']
['kansas']
['faded']
['cape']
['hippies']
['razor']
['combines']
['mutual']
['stepmother']
['betrayed']
['paycheck']
['comprehend']
['capacity']
['arc']
['captivated']
['influential']
['korea']
['excruciating']
["kubrick's"]
['abortion']
['frantic']
['cells']
['unattractive']
['distinctive']
['villagers']
['slide']
['denouement']
['produces']
['centuries']
['smug']
['barker']
['committing']
['remembering']
['nominations']
['warrant']
['pad']
['investigator']
['overboard']
['glamorous']
['arriving']
['cartoonish']
['senior']
['naval']
['toni']
['sigh']
['ebert']
['convicted']
['carnival']
['raging']
['sustain']
['flows']
['admission']
['euro']
['heath']
['dive']
['pretends']
['diamonds']
['buys']
['pizza']
['experts']
['update']
['interviewed']
["doctor's"]
['ominous']
['overbearing']
['homes']
['dynamite']
['protection']
['in

['bites']
['uptight']
['des']
['winchester']
['accompanying']
['hmm']
['peggy']
['kathleen']
['filmography']
['hepburn']
['carla']
['funds']
['bats']
['transplant']
['qualifies']
['slater']
['disappearance']
['chevy']
['informs']
['impending']
['zeta']
['fried']
['falcon']
['midway']
['pa']
['hooper']
['pompous']
['rips']
['thelma']
['thug']
['brett']
['basil']
['granger']
['succession']
['yeti']
['stadium']
['nero']
['encouraged']
['imply']
['precise']
['hercules']
['monday']
['marrying']
['anthology']
['cheaper']
['cummings']
['surrounds']
['randall']
['pigs']
['sober']
['flock']
['polly']
['josie']
['armor']
['expressing']
['decency']
['casablanca']
['shouts']
['theirs']
['photograph']
['investment']
['boxes']
['righteous']
['promoting']
['galactica']
['roland']
['perverted']
['stating']
['flame']
['sufficient']
['construct']
['worship']
['asset']
['baffling']
['lieutenant']
['loop']
['segal']
['guru']
['flee']
['continuously']
['astronaut']
["parents'"]
['sentences']
['reels']
['my